In [1]:
import pandas as pd
import numpy as np
import pyarrow
import fastparquet


import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import col,lpad, length, lit, to_date,to_timestamp, when, date_format, coalesce, md5, sha2, concat_ws,trim

In [2]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local") \
    .appName("Qlik_transfo") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/04 15:10:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/04 15:10:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
file_path_ = "/Users/dataworkshop/Desktop/Spark/Frais de gestion/Table de param & ds/datasetV1.xlsx"
file_path = "/Users/dataworkshop/Desktop/Spark/Frais de gestion/Table de param & ds/Frasi_Gestion.xlsx"
Suivi_et_Archive = "Suivi_et_Archive"
Tab_PARAM = "Tab_PARAM"
PARAM_FG = "PARAM_FG"

In [4]:
df_Suivi_et_Archive = pd.read_excel(file_path_, header=1)
df_Tab_PARAM = pd.read_excel(file_path, Tab_PARAM)
df_PARAM_FG = pd.read_excel(file_path, PARAM_FG)

Param FG

In [5]:
# effecaer les colonnes qui sont vides
columns_to_delete = ['Unnamed: 9', 'Unnamed: 12',"Unnamed: 16",]
# Effacer les colonnes vide
df_PARAM_FG = df_PARAM_FG.drop(columns=columns_to_delete)


In [6]:
schema_PARAM_FG = StructType([
    StructField('Unnamed: 0', StringType(), True),
    StructField('Année', StringType(), True),
    StructField('Catégorie', StringType(), True),
    StructField('Sous-Catégorie',StringType(), True),
    StructField('Recueillir les candidatures TTC', IntegerType(), True),
    StructField('Instruire les dossiers TTC',IntegerType(), True),
    StructField('Accompagner les projets TTC', IntegerType(), True),
    StructField('Recueillir les candidatures HT',IntegerType(), True),
    StructField('Instruire les dossiers HT', IntegerType(), True),
    StructField('Accompagner les projets HT',IntegerType(), True),
    StructField('Reporting et évaluation TTC', IntegerType(), True),
    StructField('Reporting et évaluation HT',IntegerType(), True),
    StructField('Convention SCAN', StringType(), True),
    StructField('Convention', StringType(), True),
    StructField('Sous-fonds FG AAP/Projets',StringType(), True),
    StructField('Dispositif', StringType(), True),
    StructField('Dispositif Cible', StringType(), True),
    StructField('AAP', StringType(), True),
    StructField('Date de relève',StringType(), True),
    StructField('Acronyme du projet', StringType(), True),
    StructField("Nom de l'entreprise", StringType(), True),
    StructField("Identifiant projet",StringType(), True),
    StructField("Montant total de l'aide retenue HT", IntegerType(), True),
    StructField('COPIL : Date de signature DPM',StringType(), True),
    StructField('Convention.1', StringType(), True),
    StructField('AAP_Gré-à-gré', StringType(), True),])



# MAPPING
PARAM_FG = spark.read.csv('df_PARAM_FG_clean.csv', header=True, schema=schema_PARAM_FG)

# Replace NaN values with None
PARAM_FG = PARAM_FG.na.replace(float('nan'), None)

In [7]:
PARAM_FG.show(truncate=False)  

24/01/04 15:10:23 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/01/04 15:10:24 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Année, Catégorie, Sous-Catégorie, Recueillir les candidatures TTC, Instruire les dossiers TTC, Accompagner les projets TTC, Recueillir les candidatures HT, Instruire les dossiers HT, Accompagner les projets HT, Reporting et évaluation TTC, Reporting et évaluation HT, Convention SCAN, Convention, Sous-fonds FG AAP/Projets, Dispositif, Dispositif Cible, AAP, Date de relève, Acronyme du projet, Nom de l'entreprise, Identifiant projet, Montant total de l'aide retenue HT, COPIL : Date de signature DPM, Convention.1, AAP_Gré-à-gré
 Schema: Unnamed: 0, Année, Catégorie, Sous-Catégorie, Recueillir les candidatures TTC, Instruire les dossiers TTC, Accompagner les projets TTC, Recueillir les candidatures HT, Instruir

+----------+------+---------------+--------------+-------------------------------+--------------------------+---------------------------+------------------------------+-------------------------+--------------------------+---------------------------+--------------------------+----------------------+--------------------+-------------------------+----------+----------------+--------------+--------------+------------------------------------------------+------------------------------------------+---------------------------------------+----------------------------------+-----------------------------+----------------------+------------------------------------+
|Unnamed: 0|Année |Catégorie      |Sous-Catégorie|Recueillir les candidatures TTC|Instruire les dossiers TTC|Accompagner les projets TTC|Recueillir les candidatures HT|Instruire les dossiers HT|Accompagner les projets HT|Reporting et évaluation TTC|Reporting et évaluation HT|Convention SCAN       |Convention          |Sous-fonds FG AAP/

In [8]:
df = spark.createDataFrame(pd.read_csv('dataset_version1.csv').replace({float('nan'): None}))
df.show()

/var/folders/6d/b6p57_cn0t9g6b9c6t40smg80000gn/T/ipykernel_16254/2821287331.py:1: DtypeWarning: Columns (77,78,79,80,109,112,121,122,123,124,126) have mixed types. Specify dtype option on import or set low_memory=False.
  df = spark.createDataFrame(pd.read_csv('dataset_version1.csv').replace({float('nan'): None}))
/opt/homebrew/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:
24/01/04 15:10:27 WARN TaskSetManager: Stage 1 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.


+----------+----------+-------------------+-------------------+--------------------+--------------------+------------+---------+--------------------+-----------------+----------+--------------------+---------------------------------------+---------------------------------+-----------------------------------+--------------------+---------------------+-----------------------------+------------------+------------+---------+-----------------------------------+--------------------------------------+----------------------------------------+---------------------------------------------+------------------------------------+---------------------------------------+-----------------------------------------+----------------------------------------------+--------------------+---------------------------+---------------------------------------+---------------+------------------------------+---------------------------+---------------------------+-------------+-----------------------+------------------

24/01/04 15:10:32 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 1 (TID 1): Attempting to kill Python Worker


In [9]:
Param = df_Tab_PARAM[['Statut', "Terminé/En cours",'Liste_Relèves',"Stade_Relèves_FG"]]

Param = spark.createDataFrame(Param)
# rename stat to be able to map terminé/en cours properly
Param = Param.withColumnRenamed("Statut", "Statut_")

Param.show()

/opt/homebrew/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


+--------------------+----------------+--------------------+--------------------+
|             Statut_|Terminé/En cours|       Liste_Relèves|    Stade_Relèves_FG|
+--------------------+----------------+--------------------+--------------------+
|Analyse opportuni...|        Terminés|     2021 -  févr-21|Majoritairement t...|
|Attente dépôt com...|        En cours|     2022 -  févr-22|Majoritairement t...|
| Audition: Abandonné|        Terminés|     2023 -  févr-23|Majoritairement t...|
|   Audition: Ajourné|        En cours|ACCEL CYBER AMI 1...|Majoritairement t...|
|Audition: Défavor...|        Terminés|AERONEF BAS CARBO...|Majoritairement t...|
|Audition: Réorien...|        En cours|AERONEF BAS CARBO...|Majoritairement t...|
|     COEX: Abandonné|        Terminés|AGROALIM 1 -  jan...|Majoritairement t...|
|       COEX: Ajourné|        En cours|AGROALIM 1 -  jui...|Majoritairement t...|
|   COEX: Défavorable|        Terminés|AGROALIM 1 -  mar...|Majoritairement t...|
|     COEX: Favo

lire fichier param_cadrage

In [10]:
# Specify the path to your Excel file
excel_file_path = "/Users/dataworkshop/Desktop/Spark/FG_PARAM.xlsx"

# Read the Excel file into a Pandas DataFrame
FG_Param = pd.read_excel(excel_file_path)

# Convert the Pandas DataFrame to a PySpark DataFrame
FG_Param = spark.createDataFrame(FG_Param)

FG_Param = FG_Param.withColumnRenamed("AAP", "_AAP")


# Show the content of the PySpark DataFrame
FG_Param.show()

/opt/homebrew/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


+--------------------+-------------+--------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|                _AAP|Catégorie_AAP|      Convention_AAP|AAP Actifs 2021|AAP Actifs 2022|AAP Actifs 2023|AAP Actifs 2024|AAP Actifs 2025|AAP Actifs 2026|AAP Actifs 2027|AAP Actifs 2028|AAP Actifs 2029|AAP Actifs 2030|AAP Actifs 2031|AAP Actifs 2032|AAP Actifs 2033|AAP Actifs 2034|AAP Actifs 2035|
+--------------------+-------------+--------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|AERONEF BAS CARBONE1|    SA/FR2030|Démonstrateur Dirigé|              0|              1|              1| 

In [11]:

FG_Param.show()

+--------------------+-------------+--------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|                _AAP|Catégorie_AAP|      Convention_AAP|AAP Actifs 2021|AAP Actifs 2022|AAP Actifs 2023|AAP Actifs 2024|AAP Actifs 2025|AAP Actifs 2026|AAP Actifs 2027|AAP Actifs 2028|AAP Actifs 2029|AAP Actifs 2030|AAP Actifs 2031|AAP Actifs 2032|AAP Actifs 2033|AAP Actifs 2034|AAP Actifs 2035|
+--------------------+-------------+--------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|AERONEF BAS CARBONE1|    SA/FR2030|Démonstrateur Dirigé|              0|              1|              1| 

In [12]:
from pyspark.sql import functions as F

# Les données à ramener du dataset


F_G_AAP_Phase_Cadrage = df.select("AAP", "Dispositif", "Date de relève","Identifiant projet")

# Adjust AAP Nuclaire and all AAP's

#delarer une vairable de table de param
aap_mapping_value = PARAM_FG[["Identifiant projet","AAP_Gré-à-gré"]]

# rename ID_projet to get rid of ambiguity
aap_mapping_value = aap_mapping_value.withColumnRenamed("Identifiant projet", "Identifiant projet_")



# Alias the DataFrames
df1 = F_G_AAP_Phase_Cadrage.alias("df1")
df2 = aap_mapping_value.alias("df2")

# Join with alias and specify the columns with qualified names
F_G_AAP_Phase_Cadrage = df1.join(df2, df1["Identifiant projet"] == df2["Identifiant projet_"], "left_outer")

F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn(
    "AAP",
    when(col("AAP").startswith("Gré-à-gré Etat"), col("AAP_Gré-à-gré"))
    .otherwise(F_G_AAP_Phase_Cadrage["AAP"]))

# supprimer les deux colonnes ID_projet
F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.drop("Identifiant projet")
#F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.drop("Identifiant projet_")



# Renommer date de relève

F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumnRenamed("Date de relève", "1ere année de relève")


# Mapper Catégorie AAP

FG_Param_ =FG_Param[["_AAP","Catégorie_AAP"]]

F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.join(FG_Param_, F_G_AAP_Phase_Cadrage["AAP"] == FG_Param_["_AAP"], "left_outer")
F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.drop("_AAP")

# Mapper convention 

FG_Param1 =FG_Param[["_AAP","Convention_AAP"]]

F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.join(FG_Param1, F_G_AAP_Phase_Cadrage["AAP"] == FG_Param1["_AAP"], "left_outer")
F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.drop("_AAP")
F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumnRenamed("Convention_AAP", "Convention")


# Col Cadrer la procedure - AAP renouvelé

F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn("Cadrer la procédure - AAP renouvelé", when((F_G_AAP_Phase_Cadrage["AAP"].substr(-1, 1) != "1") & (F_G_AAP_Phase_Cadrage["AAP"].substr(-1, 1) != "2"), 5500))
F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn("Cadrer la procédure - AAP renouvelé", col("Cadrer la procédure - AAP renouvelé").cast("int"))

# Col Cadrer la procedure 

F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn("Cadrer la procédure", when(F_G_AAP_Phase_Cadrage["Cadrer la procédure - AAP renouvelé"].isNull(), lit(11167)))
F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn("Cadrer la procédure", col("Cadrer la procédure").cast("int"))

# les exceptions de gré-à-gré

F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn('Cadrer la procédure', when(col('Identifiant projet_').isNotNull(), 33583).otherwise(col('Cadrer la procédure')))

F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.drop("Identifiant projet_")
F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.drop("AAP_Gré-à-gré")

# get distinct
F_G_AAP_Phase_Cadrage.distinct()

# plusieurs colonnes de Reporrting & Evaluation 

# ramener le montant qu'on va diviser
MT_Reporting_evaluation = PARAM_FG.filter(col("Reporting et évaluation HT").isNotNull())
MT_Reporting_evaluation = MT_Reporting_evaluation[["Reporting et évaluation HT"]]
MT_Reporting_evaluation = MT_Reporting_evaluation.withColumn("Reporting et évaluation HT", col("Reporting et évaluation HT").cast("int"))

# ramner les données de parametrage pour identifier les AAP sur les quel on va diviser le montant de reporting evaluation


file_path = "/Users/dataworkshop/Desktop/Spark/Table FG paramètrage frais reporting propal 2.xlsx"
AAP = "AAP"

# transform excel in to dataframe
df_reporting_annees = pd.read_excel(file_path,AAP, header=0)
param_csv =df_reporting_annees.to_csv("df_reporting_annees.csv")
df_reporting_annees = spark.createDataFrame(pd.read_csv('df_reporting_annees.csv').replace({float('nan'): None}))

df_reporting_annees.show()

# show the data

+----------+--------------------+-------------+--------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|Unnamed: 0|                 AAP|Catégorie_AAP|      Convention_AAP|AAP Actifs 2021|AAP Actifs 2022|AAP Actifs 2023|AAP Actifs 2024|AAP Actifs 2025|AAP Actifs 2026|AAP Actifs 2027|AAP Actifs 2028|AAP Actifs 2029|AAP Actifs 2030|AAP Actifs 2031|AAP Actifs 2032|AAP Actifs 2033|AAP Actifs 2034|AAP Actifs 2035|
+----------+--------------------+-------------+--------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|         0|AERONEF BAS CARBONE1|    SA/FR2030|Démonstrateur Dirigé|     

/opt/homebrew/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


Creation de toues les colonnes Reporting & Evaluation

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
import datetime
from pyspark.sql.functions import round


# Assuming you have a SparkSession named 'spark'

def calculate_result_for_year(df_reporting_annees, MT_Reporting_evaluation, year):
    # Filter rows where the column for the specific year is 1
    df_reporting_annees_year = df_reporting_annees.filter(col(f'AAP Actifs {year}') == 1)
    
    # Count the occurrences
    count_of_ones = df_reporting_annees_year.select('AAP').distinct().count()
    
    # Get the integer value from the second table
    reporting_evaluation_value = MT_Reporting_evaluation.select("Reporting et évaluation HT").collect()[0][0]
    
    # Divide the integer value by the total number of active AAPs for the specific year
    result = reporting_evaluation_value / count_of_ones
    
    # Update the DataFrame with the calculated result for the specific year
    df_reporting_annees = df_reporting_annees.withColumn(f'AAP Actifs {year}', when(col(f'AAP Actifs {year}') == 1, result).otherwise(None))
    
    return df_reporting_annees

# Get the current year
current_year = datetime.datetime.now().year

# Calculate results for each year from 2021 to the current year
for year in range(2021, current_year + 1):
    df_reporting_annees = calculate_result_for_year(df_reporting_annees, MT_Reporting_evaluation, year)

columns_to_drop = ["Unnamed: 0","Catégorie_AAP", "Convention_AAP"]
df_reporting_annees = df_reporting_annees.drop(*columns_to_drop)


# arrandire les AAP actifs

# Voici comment tu peux arrondir toutes ces colonnes
colonnes_a_arrondir = ["AAP Actifs 2022", "AAP Actifs 2023", "AAP Actifs 2024", "AAP Actifs 2025", "AAP Actifs 2026",
                       "AAP Actifs 2027", "AAP Actifs 2028", "AAP Actifs 2029", "AAP Actifs 2030", "AAP Actifs 2031",
                       "AAP Actifs 2032", "AAP Actifs 2033", "AAP Actifs 2034", "AAP Actifs 2035"]

# Appliquer la fonction round à chaque colonne
for colonne in colonnes_a_arrondir:
    df_reporting_annees = df_reporting_annees.withColumn(colonne, round(colonne, 0))



# Show the updated DataFrame
df_reporting_annees.show()

24/01/04 15:10:33 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+--------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|                 AAP|AAP Actifs 2021|AAP Actifs 2022|AAP Actifs 2023|AAP Actifs 2024|AAP Actifs 2025|AAP Actifs 2026|AAP Actifs 2027|AAP Actifs 2028|AAP Actifs 2029|AAP Actifs 2030|AAP Actifs 2031|AAP Actifs 2032|AAP Actifs 2033|AAP Actifs 2034|AAP Actifs 2035|
+--------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|AERONEF BAS CARBONE1|           NULL|        25463.0|        25463.0|        25463.0|              1|              1|              1|              1|              0|              0|              0|             

joindre la premiere partie et la deuxieme

In [14]:
df_reporting_annees = df_reporting_annees.withColumnRenamed("AAP", "AAP_")

F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.join(df_reporting_annees, F_G_AAP_Phase_Cadrage["AAP"] == df_reporting_annees["AAP_"], "left_outer")

df_reporting_annees = df_reporting_annees.drop("_AAP")

In [15]:
F_G_AAP_Phase_Cadrage.show()

24/01/04 15:10:34 WARN TaskSetManager: Stage 45 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.


+-------------------+----------+--------------------+---------------+------------------+-----------------------------------+-------------------+-------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|                AAP|Dispositif|1ere année de relève|  Catégorie_AAP|        Convention|Cadrer la procédure - AAP renouvelé|Cadrer la procédure|               AAP_|AAP Actifs 2021|AAP Actifs 2022|AAP Actifs 2023|AAP Actifs 2024|AAP Actifs 2025|AAP Actifs 2026|AAP Actifs 2027|AAP Actifs 2028|AAP Actifs 2029|AAP Actifs 2030|AAP Actifs 2031|AAP Actifs 2032|AAP Actifs 2033|AAP Actifs 2034|AAP Actifs 2035|
+-------------------+----------+--------------------+---------------+------------------+-----------------------------------+-------------------+-------------------+---------------+----------

24/01/04 15:10:38 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 45 (TID 30): Attempting to kill Python Worker


# Reporting & Evaluation TOTAL

In [16]:
from pyspark.sql.functions import col, sum, min, max, first

# convert columns to int


# Display the similar version of 

F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.groupBy("AAP").agg(
    first("Dispositif").alias("Dispositif"),
    first("Catégorie_AAP").alias("Catégorie_AAP"),
    first("Convention").alias("Convention"),
    min("1ere année de relève").alias("1ere année de relève"),
    max("Cadrer la procédure").alias("Cadrer la procédure"),
    max("Cadrer la procédure - AAP renouvelé").alias("Cadrer la procédure - AAP renouvelé"),
    first("AAP Actifs 2021").alias("Reporting & Evaluation 2021"),
    first("AAP Actifs 2022").alias("Reporting & Evaluation 2022"),
    first("AAP Actifs 2023").alias("Reporting & Evaluation 2023"),
    min("AAP Actifs 2024").alias("Reporting & Evaluation 2024"),
    min("AAP Actifs 2025").alias("Reporting & Evaluation 2025"),
    min("AAP Actifs 2026").alias("Reporting & Evaluation 2026"),
    min("AAP Actifs 2027").alias("Reporting & Evaluation 2027"),
    min("AAP Actifs 2028").alias("Reporting & Evaluation 2028"),
    min("AAP Actifs 2029").alias("Reporting & Evaluation 2029"),
    min("AAP Actifs 2030").alias("Reporting & Evaluation 2030"),
    min("AAP Actifs 2031").alias("Reporting & Evaluation 2031"),
    min("AAP Actifs 2032").alias("Reporting & Evaluation 2032"),
    min("AAP Actifs 2033").alias("Reporting & Evaluation 2033"),
    min("AAP Actifs 2034").alias("Reporting & Evaluation 2034"),
    min("AAP Actifs 2035").alias("Reporting & Evaluation 2035"),
)

F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn("Reporting & Evaluation 2021", col("Reporting & Evaluation 2021").cast("int"))
F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn("Reporting & Evaluation 2022", col("Reporting & Evaluation 2022").cast("int"))
F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn("Reporting & Evaluation 2023", col("Reporting & Evaluation 2023").cast("int"))

F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn("Cadrer la procédure", col("Cadrer la procédure").cast("int"))
F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn("Cadrer la procédure - AAP renouvelé", col("Cadrer la procédure - AAP renouvelé").cast("int"))



F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn(
    "Reporting & Evaluation TOTAL",
    F.col("Reporting & Evaluation 2021") + F.col("Reporting & Evaluation 2022") + F.col("Reporting & Evaluation 2023")
)

F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn("Reporting & Evaluation TOTAL", col("Reporting & Evaluation TOTAL").cast("int"))


F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn(
    "Total frais de gestion par AAP",
    F.coalesce(
        F.col("Reporting & Evaluation TOTAL"), 
        F.lit(0)
    ) + F.coalesce(
        F.col("Cadrer la procédure"), 
        F.lit(0)
    ) + F.coalesce(
        F.col("Cadrer la procédure - AAP renouvelé"), 
        F.lit(0)
    )
)

# Show the result


In [17]:
F_G_AAP_Phase_Cadrage.show()

24/01/04 15:10:39 WARN TaskSetManager: Stage 56 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+----------+---------------+--------------------+--------------------+-------------------+-----------------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+----------------------------+------------------------------+
|                 AAP|Dispositif|  Catégorie_AAP|          Convention|1ere année de relève|Cadrer la procédure|Cadrer la procédure - AAP renouvelé|Reporting & Evaluation 2021|Reporting & Evaluation 2022|Reporting & Evaluation 2023|Reporting & Evaluation 2024|Reporting & Evaluation 2025|Reporting & Evaluation 2026|Reporting & Evaluation 2027|Reporting & Evaluation 2028|R

24/01/04 18:31:41 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 159012 ms exceeds timeout 120000 ms
24/01/04 18:31:41 WARN SparkContext: Killing executors is not supported by current scheduler.
24/01/04 18:43:10 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at o